# 🦌 Node Menagerie ⚡

Demonstrating the variety of Node Shapes and introduce the `Symbol` classes:

- Rect - A rectangular node
- Circle - A circlular node
- Ellipse - An elliptical node
- Diamond - A diamond node
- Comment - A mostly rectangular node that has been notched in the upper right corner
- Image - Wrapper for a `image` svg tag
- Path - Wrapper for a `path` svg tag
- SVG - embed raw svg

- Widget - embed other jupyterlab widgets
- HTML - embed a html sting

These classes help to generate the valid ElkJSON structures that can be added to a
networkx graphs and passed to the Elk display widgets. The generated JSON has
`properties` -> `type` set to a value that will be inspected on the frontend to render
the appropriate shape. When calling a shapes `to_json` method an `id` arguement should
be supplied and will be included in the output dictionary.

In [1]:
import importnb

# import ipyelk.diagram.elk_export
import ipywidgets as W
import networkx as nx
from IPython.display import SVG

import ipyelk
import ipyelk.nx
import ipyelk.tools
import ipyelk.tools.tools

# from ipyelk.contrib.library import logic_gates as logic
from ipyelk.contrib.molds import connectors as conn
from ipyelk.contrib.molds import structures
from ipyelk.elements import EndpointSymbol, Symbol, SymbolSpec, shapes, layout_options as opt
from ipyelk.loaders import NXLoader

### Symbol Example

In [2]:
shapes.HTML(use="<h1>Hello World</h1>").dict().get('width') or 150

150

In [3]:
opt.NodeLabelPlacement().identifier, opt.NodeLabelPlacement().value

('org.eclipse.elk.nodeLabels.placement', 'H_LEFT V_TOP INSIDE')

In [4]:
def symbol_example():
    g = nx.Graph()
    tree = nx.DiGraph()

    width = 150
    height = 100

    collection = [
        shapes.Image(
            use="./files/examples/untitled_example.svg",
        ),
        shapes.SVG(
            use="""<g><rect fill="none" height="24" width="24"/><path d="M17,15l1.55,1.55c-0.96,1.69-3.33,3.04-5.55,3.37V11h3V9h-3V7.82C14.16,7.4,15,6.3,15,5c0-1.65-1.35-3-3-3S9,3.35,9,5 c0,1.3,0.84,2.4,2,2.82V9H8v2h3v8.92c-2.22-0.33-4.59-1.68-5.55-3.37L7,15l-4-3v3c0,3.88,4.92,7,9,7s9-3.12,9-7v-3L17,15z M12,4 c0.55,0,1,0.45,1,1s-0.45,1-1,1s-1-0.45-1-1S11.45,4,12,4z"/></g>"""
        ),
        shapes.Path(
            use="M 0,0 L 0,100 L 20,30 Z",
        ),
        shapes.ForeignObject(use="<input/>"),
        shapes.Rect(),
        shapes.Diamond(),
        shapes.Comment(),
        shapes.Circle(radius=width / 2),
        shapes.Ellipse(rx=width / 2, ry=height / 2),
        shapes.HTML(use="<h1>Hello World</h1>"),
    ]

    for i, shape in enumerate(collection):
        shape_data = shape.dict()
        data = {
            "width": str(shape_data.get("width") or width),
            "height": str(shape_data.get("height") or height),
            "properties": {"shape": shape_data},
            "labels": [{"text":shape.__class__.__name__}],
        }
        g.add_node(str(i), **data)

    # configure app
    app = ipyelk.Diagram(
        source=NXLoader(default_node_opts=None).load(
            graph=g,
            hierarchy=tree,     
        ),
        layout={"height": "100%"},
    )
    return app

In [5]:
if __name__ == "__main__":
    symbol_app = symbol_example()
    display(symbol_app)

Diagram(children=[HTML(value='<style></style>', layout=Layout(display='None')), SprottyViewer(hover=Hover(tee=…

In [6]:

# JSON(symbol_app.pipe.pipes[2].inlet.value.dict())

In [6]:
root = symbol_app.view.source.index.root
root.layoutOptions['org.eclipse.elk.algorithm']= 'org.eclipse.elk.layered'
root.layoutOptions['org.eclipse.elk.nodeLabels.placement']= 'H_LEFT V_TOP INSIDE'
root.layoutOptions.pop(opt.HierarchyHandling.identifier, None)

symbol_app.view.source._notify_trait('value', None, None)

In [7]:
from IPython.display import JSON
JSON(symbol_app.view.source.value.dict())

<IPython.core.display.JSON object>

### Widget Example

Demonstate how to attach jupyterlab widgets for use in diagram nodes.

In [8]:
def widget_example():
    slider_box = W.VBox(children=[W.FloatSlider() for i in range(5)])

    g = nx.Graph()
    tree = nx.DiGraph()

    g.add_node("sliders")

    g.add_node(
        "slider_box",
        **{
            "properties": {"shape": shapes.Widget(widget=slider_box).dict()},
            "width": 320,
            "height": 200,
        }
    )
    g.add_edge("sliders", "slider_box")

    for i, slider in enumerate(slider_box.children):
        id = str(i)
        g.add_node(
            id,
            **{
                "properties": {"shape": shapes.Widget(widget=slider).dict()},
                "width": 320,
                "height": 40,
            }
        )
        tree.add_edge("sliders", id)

    # configure app
    app = ipyelk.Diagram(
        source=NXLoader(default_node_opts=None).load(
            graph=g,
            hierarchy=tree,
        ),
        layout={"height": "100%"},
    )
#     toggle = ipyelk.tools.tools.ToggleCollapsedBtn(app=app)
#     fit = ipyelk.tools.tools.FitBtn(app=app)
#     app.toolbar.commands = [fit, toggle]
    return app, slider_box

In [9]:
if __name__ == "__main__":
    widget_app, sliders = widget_example()
    display(widget_app)

Diagram(children=[HTML(value='<style></style>', layout=Layout(display='None')), SprottyViewer(hover=Hover(tee=…

In [10]:
sliders

### Nesting Diagrams

In [11]:
def nesting_diagram_example():
    # build graphs
    slider_diagram, sliders = widget_example()
    g = nx.Graph()
    tree = nx.DiGraph()

    g.add_node("n1", width=40, height=40)
    tree.add_node("slider_diagram", hidden=True)

    g.add_node(
        "slider_diagram",
        **{
            "properties": {"shape": shapes.Widget(widget=slider_diagram).dict()},
            "width": 400,
            "height": 700,
        }
    )
    tree.add_edge("n1", "slider_diagram")

    # configure app
    app = ipyelk.Diagram(
        source=NXLoader(default_node_opts={}).load(
            graph=g,
            hierarchy=tree,
        ),
        layout={"height": "100%"},
    )
#     toggle = ipyelk.tools.tools.ToggleCollapsedBtn(app=app)
#     fit = ipyelk.tools.tools.FitBtn(app=app)
#     app.toolbar.commands = [fit, toggle]
    return app

In [12]:
if __name__ == "__main__":
    nested_app = nesting_diagram_example()
    display(nested_app)

Diagram(children=[HTML(value='<style></style>', layout=Layout(display='None')), SprottyViewer(hover=Hover(tee=…

# Label Icons

In [13]:
def label_icon_example():
    # setup icons
    conversation_def = Symbol(
        identifier="conv",
        element={
            "properties": {
                "shape": shapes.SVG(
                    use="""<path d="M24,17H8a1,1,0,0,0,0,2H24a1,1,0,0,0,0-2Z"/><path d="M24,7H8A1,1,0,0,0,8,9H24a1,1,0,0,0,0-2Z"/><path d="M24,12H8a1,1,0,0,0,0,2H24a1,1,0,0,0,0-2Z"/><path d="M25,2H7A5,5,0,0,0,2,7V27.11a3,3,0,0,0,3,3,3,3,0,0,0,1.75-.56l6.81-4.87A3,3,0,0,1,15.45,24H25a5,5,0,0,0,5-5V7A5,5,0,0,0,25,2Zm3,17a3,3,0,0,1-3,3H15.45a4.94,4.94,0,0,0-3.11,1.09L5.58,27.92a1,1,0,0,1-1,.08A1,1,0,0,1,4,27.11V7A3,3,0,0,1,7,4H25a3,3,0,0,1,3,3Z"/>"""
                )
            },
        },
        width=32,
        height=32,
    )

    class_def = Symbol(
        identifier="class",
        element={
            "properties": {
                "shape": shapes.SVG(
                    use="""<path fill="currentColor" d="M256 8C119.033 8 8 119.033 8 256s111.033 248 248 248 248-111.033 248-248S392.967 8 256 8zm0 448c-110.532 0-200-89.451-200-200 0-110.531 89.451-200 200-200 110.532 0 200 89.451 200 200 0 110.532-89.451 200-200 200zm107.351-101.064c-9.614 9.712-45.53 41.396-104.065 41.396-82.43 0-140.484-61.425-140.484-141.567 0-79.152 60.275-139.401 139.762-139.401 55.531 0 88.738 26.62 97.593 34.779a11.965 11.965 0 0 1 1.936 15.322l-18.155 28.113c-3.841 5.95-11.966 7.282-17.499 2.921-8.595-6.776-31.814-22.538-61.708-22.538-48.303 0-77.916 35.33-77.916 80.082 0 41.589 26.888 83.692 78.277 83.692 32.657 0 56.843-19.039 65.726-27.225 5.27-4.857 13.596-4.039 17.82 1.738l19.865 27.17a11.947 11.947 0 0 1-1.152 15.518z"/>"""
                ),
            }
        },
        width=496,
        height=496,
    )

    bullet_def = Symbol(
        identifier="bullet",
        element={
            "properties": {
                "shape": shapes.Circle(radius=4),
            }
        },
        width=8,
        height=8,
    )

    # build model graph
    g = nx.Graph()
    tree = nx.DiGraph()

    class_icon = shapes.Icon(use=class_def.identifier, width=12, height=12)
    conversation_icon = shapes.Icon(
        use=conversation_def.identifier, width=12, height=12
    )
    bullet_icon = shapes.Icon(use=bullet_def.identifier, width=4, height=4)

    # label location layout options
    heading_opts = opt.OptionsWidget(
        options=[
            opt.NodeLabelPlacement(horizontal="center", vertical="top"),
        ]
    ).value

    body_label_opts = opt.OptionsWidget(
        options=[
            opt.NodeLabelPlacement(horizontal="left", vertical="center"),
        ]
    ).value

    footer_label_opts = opt.OptionsWidget(
        options=[
            opt.NodeLabelPlacement(horizontal="right", vertical="bottom"),
        ]
    ).value

    heading_opts = opt.OptionsWidget(
        options=[
            opt.NodeLabelPlacement(horizontal="center", vertical="top"),
        ]
    ).value

    bullet_opts = opt.OptionsWidget(
        options=[
            opt.LabelSpacing(spacing=4),
        ]
    ).value

    # list of attributes to include in the body of the node:
    attributes = [
        "x: float",
        "y: float",
    ]

    heading = [
        {
            "id": "class_label",
            "text": "Point",
            "labels": [
                {
                    "id": "label_icon_1",
                    "properties": {
                        "shape": class_icon.dict(),
                    },
                    "width": class_icon.width,
                    "height": class_icon.height,
                    "text": " ",  # label text cannot be empty
                }
            ],
            "layoutOptions": heading_opts,
        }
    ]

    labels = heading
    for attr in attributes:
        bullet_pt = {
            "id": f"bullet_{attr}",
            "text": " ",  # label text cannot be empty
            "properties": {
                "shape": bullet_icon.dict(),
            },
            "width": bullet_icon.width,
            "height": bullet_icon.height,
            "layoutOptions": bullet_opts,
        }

        attribute_label = {
            "id": f"label_{attr}",
            "text": str(attr),
            "layoutOptions": body_label_opts,
            "labels": [bullet_pt],
            "properties": {
                "selectable": True,
            },
        }
        labels.append(attribute_label)

    # bare icon
    icon_label = {
        "id": "unique_id",
        "text": " ",  # label text cannot be empty
        "properties": {
            "shape": conversation_icon.dict(),
            "selectable": True,
        },
        "width": conversation_icon.width,
        "height": conversation_icon.height,
        "layoutOptions": footer_label_opts,
    }
    labels.append(icon_label)
    g.add_node("n1", labels=labels)
    tree.add_edge("root", "n1")

    # configure app
    app = ipyelk.from_nx(
        graph=g,
        hierarchy=tree,
        layout={"height": "100%"},
        style={
            " .hidden": {
                #             "display": "none",
            }
        },
    )
    app.view.symbols = SymbolSpec().add(
        class_def,
        conversation_def,
        bullet_def,
    )
    return app

In [14]:
if __name__ == "__main__":
    label_app = label_icon_example()
    display(label_app)

Diagram(children=[HTML(value='<style>.styled-widget-140131147727312 .hidden{}</style>', layout=Layout(display=…